
# Chatbot


### Komponensek
- **Konfiguráció**
- **Dokumentumok betöltése és chunkolása**
- **Embedding és vektortár**
- **LLM inicializálás**
- **RAG keresés és kontextus összeállítás**
- **Agent workflow**
- **Demó futtatás**


## Konfiguráció

Ez tartalmazza a különböző alapbeállításokat.  

### Paraméterek
- **`llm_model_name`**: `google/flan-t5-base` – nyílt LLM, demóra elég (pontosabb: `flan-t5-large`)  
- **`temperature`** – 0.0 -> determinisztikus válaszok
- **`embedding_model_name`**: `all-MiniLM-L6-v2` – gyors embedding model 
- **`chunk_size, chunk_overlap`** – chunk és átfedés méret
- **`top_k`** – ennyi releváns dokumentumot ad vissza a retriever
- **`max_iterations`** – agent ciklus ennyiszer futhat maximum

In [17]:
from dataclasses import dataclass
import os, random
import numpy as np

SEED = int(os.environ.get("SEED", "42"))
random.seed(SEED)
np.random.seed(SEED)

@dataclass
class Config:
    llm_model_name: str = "google/flan-t5-base"
    temperature: float = 0.0
    embedding_model_name: str = "sentence-transformers/all-MiniLM-L6-v2"
    data_dir: str = "data"
    chunk_size: int = 500
    chunk_overlap: int = 70
    top_k: int = 4
    max_iterations: int = 2

CFG = Config()

## Dokumentumok betöltése és chunkolása

Ebben a dokumentumokat olvassuk be és daraboljuk fel azokat.

- **`load_pdf_text`**: oldalanként olvassa be a PDF-ek szövegét a `pypdf` segítségével
- **`build_documents`**: minden fájlt `Document` objektummá alakít, a `metadata["source"]` mezőben megőrizve a fájlnevet
- **`chunk_documents`**: `RecursiveCharacterTextSplitter`-rel darabolja a szövegeket, a chunkolás biztosítja, hogy a modell releváns, kezelhető méretű kontextust kapjon
  



In [18]:
from pathlib import Path
from typing import List
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pypdf import PdfReader
from langchain.schema import Document

data_dir = Path(f"{CFG.data_dir}")

def load_pdf_text(path: Path) -> str:
    reader = PdfReader(str(path))
    text = ""
    for page in reader.pages:
        page_text = page.extract_text() or ""
        text += page_text + "\n"
    return text

def build_documents(data_dir: Path) -> List[Document]:
    docs = []
    for path in data_dir.glob("*"):
        if path.suffix.lower() == ".pdf":
            text = load_pdf_text(path)
        else:
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()
        if text.strip():
            docs.append(Document(page_content=text, metadata={"source": path.name}))
    return docs

def chunk_documents(docs: List[Document], chunk_size: int, overlap: int) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap, add_start_index=True)
    chunks = splitter.split_documents(docs)
    for i, d in enumerate(chunks):
        d.metadata = {**d.metadata, "chunk_id": i}
    return chunks

docs = build_documents(data_dir)
chunks = chunk_documents(docs, CFG.chunk_size, CFG.chunk_overlap)
len(docs), len(chunks), [d.metadata for d in docs]

(3,
 221,
 [{'source': 'attention_is_all_you_need.pdf'},
  {'source': 'The Secret, Magical Life Of Lithium - NOEMA.pdf'},
  {'source': 'We should fix climate change — but we should not regret it - ABC Religion & Ethics.pdf'}])

## Embedding és vektortár

Itt a feldarabolt dokumentumokat embeddingekre alakítjuk, majd egy vektortárba (FAISS) kerülnek a hatékony kereséshez.

- **`HuggingFaceEmbeddings`**: a `sentence-transformers/all-MiniLM-L6-v2` modell segítségével a szövegeket numerikus vektortérbe képezi le, ez lehetővé teszi, hogy a rendszer a tartalmi hasonlóság alapján keressen és találjon releváns szövegrészeket.
- **`FAISS`**: lokálisan futó, nagy teljesítményű vektortár.
- **`retriever`**: beállítja, hogy kereséskor hány releváns chunkot adjon vissza, ez lesz a fő komponens, amelyet a későbbi agent folyamat használ a tudás előhívására.

In [19]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

def build_vectorstore(chunks: List[Document], embedding_model_name: str) -> FAISS:
    embedder = HuggingFaceEmbeddings(model_name=embedding_model_name)
    vs = FAISS.from_documents(chunks, embedder)
    return vs

vectorstore = build_vectorstore(chunks, CFG.embedding_model_name)
retriever = vectorstore.as_retriever(search_kwargs={"k": CFG.top_k})

## LLM inicializálás

Ebben betöltünk egy nyílt forrású LLM-et a HuggingFace `transformers` könyvtárából, majd egy egyszerű `generate` függvényt definiálunk.

- **`make_llm`**  
  - `AutoTokenizer` és `AutoModelForSeq2SeqLM` segítségével betölti a választott modellt
  - `pipeline("text2text-generation")` formátumban ad vissza egy használható generáló objektumot.  

- **`generate`**  
  - egyetlen szöveges promptból választ ad
  - `max_new_tokens`: maximális válasz-hossz 
  - `temperature`: befolyásolja a determinisztikusságot  

In [20]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

def make_llm(model_name: str):
    tok = AutoTokenizer.from_pretrained(model_name)
    mdl = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    pipe = pipeline("text2text-generation", model=mdl, tokenizer=tok)
    return pipe

llm = make_llm(CFG.llm_model_name)

def generate(prompt: str, max_new_tokens: int = 256, temperature: float = CFG.temperature) -> str:
    out = llm(prompt, max_new_tokens=max_new_tokens, do_sample=temperature>0.0, temperature=temperature)[0]["generated_text"]
    return out.strip()


Device set to use cpu


## RAG keresés és kontextus összeállítás

Ebben a lépésben a retriever segítségével releváns dokumentumokat keresünk, majd azokból összeállítunk egy kontextus-blokkot, amelyet a nyelvi modellnek adunk át.

- **`rag_search`**
  - A felhasználói lekérdezést lefuttatja a vektortárban.
  - Visszaad egy listát a legrelevánsabb `Document` objektumokból.

- **`build_context`**
  - A talált dokumentumokat formázott szövegblokká alakítja.
  - Tartalmazza:
    - sorszámot (`[i]`),
    - forrásfájlt (`src`),
    - chunk azonosítót (`chunk`)
  - Ez biztosítja, hogy az LLM lássa a szöveget és az eredetére vonatkozó metaadatokat.

In [21]:
def rag_search(query: str) -> List[Document]:
    return retriever.get_relevant_documents(query)

def build_context(docs: List[Document]) -> str:
    blocks = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("source", "unknown")
        cid = d.metadata.get("chunk_id", i)
        blocks.append(f"[{i}] (src={src}, chunk={cid})\n{d.page_content}")
    return "\n\n".join(blocks)

## Agent workflow

Itt hozzuk létre a LangGraph alapú agentet.

### Állapot (`AgentState`)
Az agent állapotát egy `TypedDict` írja le, ez tartalmazza többek között:
- **`question`** – a felhasználó kérdése  
- **`plan`**, **`step`** – tervezés és lépéskövetés  
- **`retrieved`**, **`context`** – talált dokumentumok és a kontextus  
- **`answer`**, **`citations`** – generált válasz és hivatkozások  
- **`iterations`**, **`score`**, **`logs`** – önreflexió és naplózás  

### Fő csomópontok
- **`planner_node`**: alapértelmezett tervet ad (search → synthesize → reflect).  
- **`search_node`**: keresést futtat, kontextust épít.  
- **`synthesize_node`**: LLM-mel választ generál, hivatkozásokkal.  
- **`reflect_node`**: pontozza a választ egyszerű logika alapján (forráshivatkozások és hossz).  

### Vezérlés
- **`controller`**: ha az értékelés < 0.55 és még nem érte el a max iterációt, újrakérdez → vissza a *search* lépéshez, különben az agent lezárja a folyamatot.

In [22]:
from typing import Any, Dict, TypedDict
from langgraph.graph import StateGraph, END

class AgentState(TypedDict):
    question: str
    plan: List[str]
    step: int
    retrieved: List[Document]
    context: str
    answer: str
    citations: List[Dict[str, Any]]
    iterations: int
    logs: List[str]
    score: float

ANSWER_PROMPT = """You are a helpful assistant.
Read the context and answer the question in 1–2 complete sentences.
Question: {question}
Context:
{context}

Answer:"""

def planner_node(state: AgentState) -> AgentState:
    if not state.get("plan"):
        state["plan"] = ["search", "synthesize", "reflect"]
        state["step"] = 0
        state.setdefault("iterations", 0)
        state.setdefault("logs", []).append("Planner: plan: search → synthesize → reflect")
    return state

def search_node(state: AgentState) -> AgentState:
    q = state["question"]
    docs = rag_search(q)
    state["retrieved"] = docs
    state["context"] = build_context(docs)
    state.setdefault("logs", []).append(f"Search: {len(docs)} relevant snippets")
    return state

def synthesize_node(state: AgentState) -> AgentState:
    prompt = ANSWER_PROMPT.format(question=state["question"], context=state.get("context", ""))
    ans = generate(prompt, max_new_tokens=320)
    citations = []
    for i, d in enumerate(state.get("retrieved", []), 1):
        citations.append({"ref": i, "source": d.metadata.get("source", "unknown"), "chunk_id": d.metadata.get("chunk_id")})
    state["answer"] = ans
    state["citations"] = citations
    state.setdefault("logs", []).append("Synthesize: answer generated")
    return state

def reflect_score(state: AgentState) -> float:
    ans = state.get("answer", "")
    srcs = len(state.get("citations", []))
    length_bonus = min(len(ans) / 400, 1.0)
    score = 0.5 * (srcs > 0) + 0.5 * length_bonus
    return float(score)

def reflect_node(state: AgentState) -> AgentState:
    score = reflect_score(state)
    state["score"] = score
    state["iterations"] = state.get("iterations", 0) + 1
    state.setdefault("logs", []).append(f"Reflect: score={score:.2f}, iter={state['iterations']}")
    return state

def controller(state: AgentState) -> str:
    if state.get("score", 0) < 0.55 and state.get("iterations", 0) < CFG.max_iterations:
        refined = state["question"] + " Please explain more deeply and give a definition."
        state["question"] = refined
        state.setdefault("logs", []).append("Controller: low score → another round of search and synthesis")
        return "search"
    else:
        state.setdefault("logs", []).append("Controller: adequate score or limit reached → END")
        return "end"

workflow = StateGraph(AgentState)
workflow.add_node("planner", planner_node)
workflow.add_node("search", search_node)
workflow.add_node("synthesize", synthesize_node)
workflow.add_node("reflect", reflect_node)

workflow.set_entry_point("planner")
workflow.add_edge("planner", "search")
workflow.add_edge("search", "synthesize")
workflow.add_edge("synthesize", "reflect")
workflow.add_conditional_edges("reflect", controller, {"search": "search", "end": END})

app = workflow.compile()

## Demó futtatás

In [23]:

def run_agent(question: str) -> Dict[str, Any]:
    state: AgentState = {"question": question}
    result = app.invoke(state)
    return result

demo_questions = [
    "What does embedding mean?",
    "What is the essence of attention?",
    "What can lithium be used for?",
    "Who invented the battery and when?",
    "What has the most impact on climate change?",
]

for q in demo_questions:
    out = run_agent(q)
    print("KÉRDÉS:", q)
    print("\nVÁLASZ:\n", out.get("answer", ""))
    print("\nHIVATKOZÁSOK:", out.get("citations", []))
    print("\nNAPLÓ:")
    for log in out.get("logs", []):
        print(" -", log)


Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors


KÉRDÉS: What does embedding mean?

VÁLASZ:
 the same across different positions, they use different parameters from layer to layer. Another way of describing this is as two convolutions with kernel size 1. The dimensionality of input and output is dmodel = 512, and the inner-layer has dimensionality dff = 2048

HIVATKOZÁSOK: [{'ref': 1, 'source': 'attention_is_all_you_need.pdf', 'chunk_id': 33}, {'ref': 2, 'source': 'attention_is_all_you_need.pdf', 'chunk_id': 35}, {'ref': 3, 'source': 'attention_is_all_you_need.pdf', 'chunk_id': 50}, {'ref': 4, 'source': 'attention_is_all_you_need.pdf', 'chunk_id': 31}]

NAPLÓ:
 - Planner: plan: search → synthesize → reflect
 - Search: 4 relevant snippets
 - Synthesize: answer generated
 - Reflect: score=0.83, iter=1
 - Controller: adequate score or limit reached → END
KÉRDÉS: What is the essence of attention?

VÁLASZ:
 relating different positions of a single sequence in order to compute a representation of the sequence

HIVATKOZÁSOK: [{'ref': 1, 'so

## Bottleneckek, teljesítménymérési és továbbfejlesztési ötletek

### Jelenlegi bottleneckek
- **LLM**: a `flan-t5-base` modell nem feltétlen ad megfelelő válaszokat, egy nagyobb modellel jobb eredményeket lehetne elérni
- **Chunkolás**: fix méretű chunkolásnál előfordulhat, hogy fontos információ szétszakad vagy kimarad.
- **Egyszerű minőségértékelés**: a `reflect_score` csak hosszt és forráshivatkozást figyel, lehetne pontosabban értékelni
- **Agentic loop**: az újra futás nem feltétlen javít az eredményen
- **Agent tervezés**: a `planner_node` mindig ugyanazt az alapértelmezett tervet adja. Nincs valódi adaptivitás a felhasználói kérdéshez vagy a keresés eredményeihez.

### Teljesítménymérési ötletek
- **Retrieval hatékonyság**: Megállapítani, hogy a releváns találatok hanyad része kerül be a top-k listába.  
- **Generált válaszok**: LLM‑alapú minőségellenőrzés
- **Latency**: Keresés és generálás időmérése külön-külön.  

### Továbbfejlesztési ötletek
- **LLM**: jobb modell a pontosabb válaszokhoz
- **Dinamikus chunkolás**: bekezdés- vagy mondat-alapú darabolás a fix karakterhossz helyett
- **Fejlettebb értékelés**: LLM-alapú reflexió, logikai következetesség ellenőrzése
- **Bővített agent logika**: több eszköz, kifinomultabb vezérlés